In [1]:
import Text.Show.Pretty
import Prelude hiding (lookup)
import Data.Bits

type Key = Int

data BinaryTrie a
    = BTNil
    | BTTip a
    | BTBin (BinaryTrie a) (BinaryTrie a)
    deriving (Show)

In [2]:
lookupBT :: Key -> BinaryTrie a -> Maybe a
lookupBT _ BTNil = Nothing
lookupBT _ (BTTip v) = Just v
lookupBT k (BTBin l r) = lookupBT (k `div` 2) (if even k then l else r)

In [3]:
-- {1 -> "x", 4 -> "y", 5 -> "z"}

fig1a =
    BTBin
        (BTBin
            (BTBin BTNil (BTTip "y"))
            (BTBin BTNil BTNil))
        (BTBin
            (BTBin (BTTip "x") (BTTip "z"))
            (BTBin BTNil BTNil))

btBin :: BinaryTrie a -> BinaryTrie a -> BinaryTrie a
btBin BTNil BTNil = BTNil
btBin l r = BTBin l r

fig1b = go fig1a
    where
        go (BTBin l r) = btBin (go l) (go r)
        go o = o

pPrint fig1b

BTBin
  (BTBin (BTBin BTNil (BTTip "y")) BTNil)
  (BTBin (BTBin (BTTip "x") (BTTip "z")) BTNil)

In [4]:
data BinaryTrie' a
    = BTNil'
    | BTTip' Int a
    | BTBin' Int (BinaryTrie' a) (BinaryTrie' a)
    deriving (Show)

lookupBT' :: Key -> BinaryTrie' a -> Maybe a
lookupBT' k BTNil' = Nothing
lookupBT' k (BTTip' j a) = if k==j then Just a else Nothing
lookupBT' k (BTBin' m l r) = if zeroBit k m
    then lookupBT' k l
    else lookupBT' k r

zeroBit :: Int -> Int -> Bool
zeroBit k m = k .&. m == 0

btBin' :: Int -> BinaryTrie' a -> BinaryTrie' a -> BinaryTrie' a
btBin' _ BTNil' t@BTTip'{} = t
btBin' _ t@BTTip'{} BTNil' = t
btBin' m l r = BTBin' m l r

In [5]:
data PatriciaTree a
    = PTNil
    | PTTip Int a
    | PTBin Int Int (PatriciaTree a) (PatriciaTree a)
    deriving (Show)

lookupPT :: Key -> PatriciaTree a -> Maybe a
lookupPT k PTNil = Nothing
lookupPT k (PTTip j a) = if k==j then Just a else Nothing
lookupPT k (PTBin p m l r) = if not (matchPrefix k p m) then Nothing
    else if zeroBit k m
        then lookupPT k l
        else lookupPT k r

mask :: Key -> Int -> Int
mask k m = k .&. (m-1)

matchPrefix :: Key -> Int -> Int -> Bool
matchPrefix k p m = mask k m == p

ptBin :: Int -> Int -> PatriciaTree a -> PatriciaTree a -> PatriciaTree a
ptBin p m PTNil t = t
ptBin p m t PTNil = t
ptBin p m l r     = PTBin p m l r

In [6]:
join :: Int -> PatriciaTree a -> Int -> PatriciaTree a -> PatriciaTree a
join p0 t0 p1 t1 = let
    m = branchingBit p0 p1
    in if zeroBit p0 m
        then PTBin (mask p0 m) m t0 t1
        else PTBin (mask p0 m) m t1 t0

branchingBit :: Int -> Int -> Int
branchingBit a b = lowestBit (xor a b)

lowestBit :: Int -> Int
lowestBit x = x .&. (negate x)

In [7]:
insert :: (a -> a -> a) -> Key -> a -> PatriciaTree a -> PatriciaTree a
insert c k x = ins
    where
        ins PTNil = PTTip k x
        ins t@(PTTip j y) = if j == k
            then PTTip k (c x y)
            else join k (PTTip k x) j t
        ins t@(PTBin p m l r) = if matchPrefix k p m
            then if zeroBit k m
                then PTBin p m (ins l) r
                else PTBin p m l (ins r)
            else join k (PTTip k x) p t

In [8]:
merge :: (a -> a -> a) -> PatriciaTree a -> PatriciaTree a -> PatriciaTree a
merge c = mrg
    where
        mrg PTNil t = t
        mrg t PTNil = t
        mrg (PTTip k x) t = insert c k x t
        mrg t (PTTip k x) = insert (flip c) k x t
        mrg s@(PTBin p m s0 s1) t@(PTBin q n t0 t1)
            {- The trees have the same prefix. Merge the subtrees. -}
            | m == n && p == q = PTBin p m (mrg s0 t0) (mrg s1 t1)
            {- q contains p. Merge t with a subtree of s. -}
            | m < n && matchPrefix q p m = if zeroBit q m
                then PTBin p m (mrg s0 t) s1
                else PTBin p m s0 (mrg s1 t)
            {- p contains q. Merge s with a subtree of t. -}
            | m > n && matchPrefix p q n = if zeroBit p n
                then PTBin q n (mrg s t0) t1
                else PTBin q n t0 (mrg s t1)
            {- The prefixes disagree. -}
            | otherwise = join p s q t

In [9]:
fromList :: [(Int, a)] -> PatriciaTree a
fromList = foldr (uncurry (insert const)) PTNil

t = fromList [(1, "x"), (5, "z"), (4, "y")]
t

PTBin 0 1 (PTTip 4 "y") (PTBin 1 4 (PTTip 1 "x") (PTTip 5 "z"))